# Load env and files

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fossil_classification import *
from enrich_holdings import *
from reports_etl import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [4]:
all_holdings_cls_path = "data/downloaded reports/company reports/all_holdings_with_cls.csv"
all_holdings_cls = pd.read_csv(all_holdings_cls_path, dtype=holdings_dtypes())

tlv_s2i = prepare_tlv_sec_num_to_issuer(fetch_latest_tlv_sec_num_to_issuer())
isin2lei = fetch_latest_isin2lei()

/Users/urimarom/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,13,14,20,21,22,23,24,25,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# adding "I_" to ParentCorpLegalId to avoid confusion with il_corp_num
all_holdings_cls["ParentCorpLegalId"] = "I_" + all_holdings_cls["ParentCorpLegalId"]

In [6]:
# enrich holdings file - fix IDs
all_holdings_cls = add_all_id_types_to_holdings(all_holdings_cls, tlv_s2i, isin2lei)

/Users/urimarom/anaconda3/lib/python3.6/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)



Holding file ISIN col is: מספר ני"ע
number of ISINs: 79957 out of 509680 rows

Holding file מספר תאגיד col is: מספר מנפיק
number of מספר תאגידs: 168824 out of 509680 rows
מספר ני"עs with matching ISIN: 175219 out of total relevant rows: 427612
מספר תאגידs with matching מספר מנפיק: 187999 out of total relevant rows: 168824
מספר ני"עs with matching מספר מנפיק: 189658 out of total relevant rows: 427612
ISINs with matching מספר מנפיק: 190070 out of total relevant rows: 175219
ISINs with matching LEI: 42776 out of total relevant rows: 175219


# Let the Analysis begin!

In [7]:
pd.DataFrame(all_holdings_cls.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"2,358,320,400.84"
גמל,"3,280,734,368.99"
פנסיה,"3,123,408,242.73"


In [8]:
def get_summary(arg1, *args, holdings=all_holdings_cls):
    '''get summary stats grouped by 1 or more columns, e.g. Company, holding_type
    '''
    group = [arg1]+[*args]
    summary = pd.DataFrame(holdings.groupby(group, dropna=False).agg(
    {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    summary["שיעור פוסילי מסך הנכסים"] = 100 * summary["שווי פוסילי"] / summary["שווי"]
    # add summary of stocks and bonds only
    holdings_cls_trd_stocks_bonds = holdings[holdings["holding_type"].isin(['מניות', 'אג"ח קונצרני'])]
    summary_tradable_stocks_bonds_only = pd.DataFrame(holdings_cls_trd_stocks_bonds.groupby(group, dropna=False).agg(
    {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    summary_tradable_stocks_bonds_only["שיעור פוסילי במניות ואגח קונצרני סחירים"] = 100 * summary_tradable_stocks_bonds_only[
        "שווי פוסילי"] / summary_tradable_stocks_bonds_only["שווי"]
    summary = summary.merge(summary_tradable_stocks_bonds_only,
                            left_on=group,
                            right_on=group,
                            how='left',
                            suffixes = ['', ' במניות ואגח קונצרני סחירים']
                           )
    return summary

## Get Company level stats over time

### Filter results to include only major companies

In [9]:
institutions = [
    'מנורה',
    'מגדל',
    'הראל',
    'מיטב דש',
    'אלטשולר',
    'מור',
    'הפניקס',
    'כלל',
    'פסגות',
    'הלמן',
    'אנליסט',
    'ילין'
]

mask = all_holdings_cls["ParentCorpName"].str.startswith(tuple(institutions))

# removing הלמן-אלדובי חח"י גמל בע"מ 515447035 & 520027715, מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל בע"מ
all_holdings_cls_filtered = all_holdings_cls[mask][~all_holdings_cls[mask]["ParentCorpLegalId"].isin(['520027715', '515447035'])]
# all_holdings_cls_filtered.groupby(["ParentCorpName","ParentCorpLegalId"]).size()

all_holdings_cls_filtered['ParentCorpGroup'] = all_holdings_cls_filtered['ParentCorpName'].str.split().str[0]
all_holdings_cls_filtered['ReportPeriodDate'] = all_holdings_cls_filtered['ReportPeriodDesc'].map(report_period_desc_to_date)
company_stats_since_2020 = get_summary('ReportPeriodDate', 'ParentCorpGroup', holdings=all_holdings_cls_filtered)

In [22]:
company_stats_since_2020.to_csv(response_path+"company_stats_since_2020.csv", index=False)

## Export holdings with missing classification that should be classified

In [28]:
majors_missing_cls = all_holdings_cls_filtered[
    (all_holdings_cls_filtered["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls_filtered["is_fossil"].isnull())
]

cols = ['שם המנפיק/שם נייר ערך' ,'מספר ני"ע' ,'מספר מנפיק']
missing_cls_deduped = majors_missing_cls[cols].drop_duplicates()

In [52]:
missing_cls_deduped.to_csv(response_path+"missing_cls.csv", index=False)

## Compare Fossil Holdings over quarters

In [10]:
all_holdings_cls_filtered.head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,ParentCorpGroup,ReportPeriodDate
433,עו'ש - בנק U-BANK,26- 1111111111,26,AA-,פנימי,שקל חדש,0.00,0.00,"328,727.39",0.05,0.01,1828503,מזומנים,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,None,None,NaN,הפניקס,2020-03-31
434,עו'ש - בנק דיסקונט,11- 1111111111,11,ILAA+,S&P מעלות,שקל חדש,0.00,0.00,"4,344.55",0.00,0.00,1828503,מזומנים,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,None,None,NaN,הפניקס,2020-03-31
435,עו'ש - בנק הבינלאומי,31- 1111111111,31,ILAA+,S&P מעלות,שקל חדש,0.00,0.00,83.55,0.00,0.00,1828503,מזומנים,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,None,None,NaN,הפניקס,2020-03-31
436,עו'ש - בנק הפועלים,12- 1111111111,12,ILAAA,S&P מעלות,שקל חדש,0.00,0.00,"120,490.05",0.02,0.00,1828503,מזומנים,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,None,None,NaN,הפניקס,2020-03-31
437,עו'ש - בנק לאומי,10- 1111111111,10,ILAAA,S&P מעלות,שקל חדש,0.00,0.00,"369,950.77",0.06,0.01,1828503,מזומנים,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,None,None,NaN,הפניקס,2020-03-31


In [59]:
quarters_for_comparison = ['2021 רבעון 2', '2021 רבעון 3']
company_for_comparison = ['מגדל']

majors_fossil_holdings_for_comparison = all_holdings_cls_filtered[
    (all_holdings_cls_filtered["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls_filtered["is_fossil"] == 1) &
    (all_holdings_cls_filtered["ReportPeriodDesc"].isin(quarters_for_comparison)) &
    (all_holdings_cls_filtered["ParentCorpGroup"].isin(company_for_comparison))
]

majors_fossil_holdings_for_comparison["שווי פוסילי"].sum()

18217561.370134484

## group holdings for summary stats per held company

### Handle Israeli holdings (having Israeli security num)

In [60]:
il_fossil_holdings = majors_fossil_holdings_for_comparison.groupby(["ParentCorpGroup", "ReportPeriodDate", 'מספר ני"ע'])
il_fossil_holdings_agg = il_fossil_holdings.agg(
    name=pd.NamedAgg(column="שם המנפיק/שם נייר ערך", aggfunc="first"),
    issuer_num=pd.NamedAgg(column="מספר מנפיק", aggfunc="first"),
    il_corp_num=pd.NamedAgg(column="מספר תאגיד", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="שווי פוסילי", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="ערך נקוב", aggfunc="sum")
).reset_index()
il_fossil_holdings_agg.head()

,ParentCorpGroup,ReportPeriodDate,"מספר ני""ע",name,issuer_num,il_corp_num,fossil_sum,quantity_sum
0,מגדל,2021-06-30,1081942,שיכון ובינוי,1068,520036104,"327,894.83","15,554,783.01"
1,מגדל,2021-06-30,1084128,"דלק קבוצה- קבוצת דלק בע""מ",1095,520044322,4.15,19.00
2,מגדל,2021-06-30,1093202,"דור אלון- דור אלון אנרגיה בישראל (1988) בע""מ",1072,520043878,2.23,24.00
3,מגדל,2021-06-30,1100007,פז נפט*,1363,510216054,"328,987.61","822,058.01"
4,מגדל,2021-06-30,1129493,"תומר תמלוגי אנרגיה (2012) בע""מ*",1609,514837111,"10,688.16","1,590,500.02"


In [61]:
il_fossil_holdings_by_issuer = il_fossil_holdings_agg.groupby(["ParentCorpGroup", "ReportPeriodDate", 'issuer_num'])
il_fossil_holdings_by_issuer_agg = il_fossil_holdings_by_issuer.agg(
    name=pd.NamedAgg(column="name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

In [62]:
il_fossil_holdings_by_issuer_agg["fossil_sum"].sum()

13884483.2273929

In [63]:
majors_fossil_holdings_for_comparison[majors_fossil_holdings_for_comparison['מספר ני"ע'].notnull()]["שווי פוסילי"].sum()

13884483.2273929

Great Success!

### Handle non-Israeli holdings

In [92]:
non_il_fossil_holdings = majors_fossil_holdings_for_comparison[majors_fossil_holdings_for_comparison['מספר ני"ע'].isnull()]
non_il_fossil_holdings["שווי פוסילי"].sum()

4333078.14274158

In [93]:
non_il_fossil_holdings["ISIN"].isnull().sum()

0

In [94]:
non_il_fossil_holdings = non_il_fossil_holdings.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'ISIN'
])
non_il_fossil_holdings_agg = non_il_fossil_holdings.agg(
    name=pd.NamedAgg(column="שם המנפיק/שם נייר ערך", aggfunc="first"),
    issuer_num=pd.NamedAgg(column="מספר מנפיק", aggfunc="first"),
    lei=pd.NamedAgg(column="LEI", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="שווי פוסילי", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="ערך נקוב", aggfunc="sum")
).reset_index()

print(non_il_fossil_holdings_agg["fossil_sum"].sum())
# Fill in issuer_num for missing LEIs
non_il_fossil_holdings_agg["issuer_num"] = non_il_fossil_holdings_agg["issuer_num"].fillna(non_il_fossil_holdings_agg["lei"])
non_il_fossil_holdings_agg.head()

4333078.14274158


,ParentCorpGroup,ReportPeriodDate,ISIN,name,issuer_num,lei,fossil_sum,quantity_sum
0,מגדל,2021-06-30,CA29250N1050,ENBRIDGE INC- ENBRIDGE,27509,None,14.36,110.00
1,מגדל,2021-06-30,FR0011401728,EDF 6 PREP 01/26,10781,549300X3UK4GG3FNMO06,"122,250.19","23,700,000.01"
2,מגדל,2021-06-30,FR0011697028,Electricite De Franc 5 01/26,27129,549300X3UK4GG3FNMO06,"44,318.53","10,000,000.00"
3,מגדל,2021-06-30,FR0013534336,Electricite De France 3.375,27129,549300X3UK4GG3FNMO06,"41,455.83","10,000,000.00"
4,מגדל,2021-06-30,GB00B1XZS820,ANGLO AMERICAN,27083,549300S9XF92D1X8ME43,"126,486.95","974,694.01"


In [95]:
non_il_fossil_holdings_by_issuer = non_il_fossil_holdings_agg.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'issuer_num'
], dropna=False)
non_il_fossil_holdings_by_issuer_agg = non_il_fossil_holdings_by_issuer.agg(
    name=pd.NamedAgg(column="name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

In [96]:
non_il_fossil_holdings_by_issuer_agg

,ParentCorpGroup,ReportPeriodDate,issuer_num,name,fossil_sum,quantity_sum
0,מגדל,2021-06-30,10056,BPLN 4.875 PERP 03/30,"131,935.78","37,000,000.00"
1,מגדל,2021-06-30,10137,NRG ENERGY- ENERGY SELECT,5.26,40.00
2,מגדל,2021-06-30,10139,ENI 3.375 PERP,"122,822.12","29,200,000.00"
3,מגדל,2021-06-30,10632,MARATHON OIL CORP- MARATHON OIL CORP,10.92,246.00
4,מגדל,2021-06-30,10781,EDF 6 PREP 01/26,"122,250.19","23,700,000.01"
5,מגדל,2021-06-30,11063,PEABODY ENERGY- PEABODY ENERGY CORP,27.40,"1,060.00"
6,מגדל,2021-06-30,11068,GLENCORE,"106,947.17","7,649,999.96"
7,מגדל,2021-06-30,11139,SSE SSELN 4.75 9/77 06/22,"137,686.53","40,296,000.02"
8,מגדל,2021-06-30,1762,ENERGEAN OIL & GAS,"128,002.76","3,968,292.99"
9,מגדל,2021-06-30,27083,ANGLO AMERICAN,"126,486.95","974,694.01"


In [89]:
non_il_fossil_holdings_by_issuer_agg["fossil_sum"].sum()

4333078.14274158

### add Israeli and non-Israeli holdings

In [90]:
fossil_holdings_by_issuer_agg = pd.concat([il_fossil_holdings_by_issuer_agg, non_il_fossil_holdings_by_issuer_agg])
fossil_holdings_by_issuer_agg["fossil_sum"].sum()

18217561.37013448

In [97]:
fossil_holdings_by_issue_grp = fossil_holdings_by_issuer_agg.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'issuer_num'
], dropna=False)
fossil_holdings_by_issuer_agg = fossil_holdings_by_issue_grp.agg(
    name=pd.NamedAgg(column="name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

In [100]:
fossil_holdings_by_issuer_agg.sort_values(["issuer_num", "name", "ReportPeriodDate"])

,ParentCorpGroup,ReportPeriodDate,issuer_num,name,fossil_sum,quantity_sum
0,מגדל,2021-06-30,10139,ENI 3.375 PERP,"122,822.12","29,200,000.00"
44,מגדל,2021-09-30,10139,ENI 3.375 PERP- Eni S.P.A,"119,742.64","29,200,000.00"
1,מגדל,2021-06-30,1068,שיכון ובינוי,"327,894.83","15,554,783.01"
45,מגדל,2021-09-30,1068,"שיכון ובינוי- שיכון ובינוי בע""מ","309,531.17","16,291,113.99"
2,מגדל,2021-06-30,1072,"דור אלון- דור אלון אנרגיה בישראל (1988) בע""מ",2.23,24.00
46,מגדל,2021-09-30,1072,"דור אלון- דור אלון אנרגיה בישראל (1988) בע""מ",2.15,24.00
47,מגדל,2021-09-30,10743,*ARCELORMITTAL- ARCELORMITTAL,"103,797.29","1,050,000.01"
3,מגדל,2021-06-30,10758801,אנרגיאן נפט וגז,"39,347.01","1,197,778.00"
4,מגדל,2021-06-30,1095,"דלק קבוצה- קבוצת דלק בע""מ",12.82,"8,753.00"
48,מגדל,2021-09-30,1095,"דלק קבוצה- קבוצת דלק בע""מ",19.61,"10,105.00"


# Next Steps
1. clean company names - function for fossil classification. Use also here.
2. finish fossil holdings comparison. Share results. A list of all fossil holdings (full outer join), then compute changes between quarters.